In [1]:
import pandas as pd
import numpy as np  

### Get matches from patch >= 54 (latest patch for captains draft)

In [2]:
# metadata of matches dataframe
cols_to_read = [
    'match_id', 'duration', 'radiant_win', 'game_mode',
    'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 
    'barracks_status_dire', 'first_blood_time', 'radiant_score', 'dire_score',
    'radiant_team_id', 'dire_team_id', 'throw', 'loss', 'comeback', 'stomp', 'patch'
]
def read_metadata(csv_paths):
    # List to store DataFrames
    df_list = []
    
    for csv_path in csv_paths:
        # Read the CSV file
        df = pd.read_csv(csv_path, usecols=cols_to_read, index_col='match_id')
        
        # Filter rows based on 'patch' value and only captains mode
        df = df[(df['patch'] >= 53) & (df['game_mode'] == 2)]
        
        # Convert to decimal the status of barracks and tower status
        # Uncomment and modify the following lines if needed
        # df[['barracks_status_dire', 'barracks_status_radiant', 
        #     'tower_status_dire', 'tower_status_radiant']] = df[['barracks_status_dire','barracks_status_radiant' ,
        #                                                         'tower_status_dire','tower_status_radiant']].map(
        #     lambda x: int(str(x).strip().replace("'", ""), 2)
        # )
        
        # Fill NaN values with 0 in specified columns
        df[['throw', 'loss', 'comeback', 'stomp']] = df[['throw', 'loss', 'comeback', 'stomp']].fillna(0)
        
        # Drop the 'patch' column
        df = df.drop(columns=['patch'])
        
        # Append the DataFrame to the list
        df_list.append(df)
    
    # Concatenate all DataFrames in the list into a single DataFrame
    final_df = pd.concat(df_list)
    

    return final_df

In [3]:
# generate metadata for reference
patch_folders = ['2023', '202401', '202402', '202403', '202404', '202405', '202406', '202407', '202408']
patch_to_read = [f'./data/{patch}/main_metadata.csv/main_metadata.csv' for patch in patch_folders]

dota_df = read_metadata(patch_to_read)
dota_df

,barracks_status_dire,barracks_status_radiant,dire_score,duration,first_blood_time,game_mode,radiant_score,radiant_win,tower_status_dire,tower_status_radiant,throw,loss,comeback,stomp,dire_team_id,radiant_team_id
match_id,,,,,,,,,,,,,,,,
7276712204,'00111111','00111100',51,1792,43,2,27,False,'0000011111110110','0000011100100000',0.0,0.0,2981.0,18902.0,NaN,NaN
7276715296,'00111111','00110011',21,1720,12,2,12,False,'0000011111110100','0000011100000110',0.0,0.0,648.0,11154.0,NaN,NaN
7276728072,'00111100','00111111',20,1627,59,2,28,True,'0000011111110000','0000011110110111',4705.0,12133.0,0.0,0.0,NaN,NaN
7276741510,'00111111','00110011',31,1354,27,2,19,False,'0000011111110110','0000011100000110',0.0,0.0,790.0,12007.0,NaN,NaN
7276741939,'00111111','00110000',36,2155,264,2,26,False,'0000011100000100','0000011000000000',0.0,0.0,6310.0,7296.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7903694984,'00111100','00111111',26,2039,173,2,36,True,'0000011100000000','0000011100110110',5320.0,10073.0,0.0,0.0,8970060.0,8849990.0
7903721670,'00111111','00000011',32,1797,170,2,14,False,'0000011110110110','0000000000000100',0.0,0.0,1364.0,15735.0,9367009.0,9081369.0
7903730404,'00110011','00111111',26,1968,145,2,33,True,'0000000110000100','0000011110110110',5468.0,10916.0,0.0,0.0,8849990.0,8970060.0


In [4]:
# check if all matches unique
len(dota_df.index.unique())

29653

### Generate reference dataframes

In [5]:
# merge all pick_ban csv
def read_draft(csv_paths, matches):
    all_picks_bans = []  # List to store DataFrames from each CSV file
    
    for csv_path in csv_paths:
        # Read each CSV file
        picks_bans = pd.read_csv(csv_path)
        
        # Filter by match_id and drop unnecessary columns
        picks_bans = picks_bans[picks_bans['match_id'].isin(matches.index)]
        picks_bans = picks_bans.drop([picks_bans.columns[0], 'ord', 'leagueid', 'is_pick'], axis=1, errors='ignore').reset_index(drop=True)
        
        # Append to the list
        all_picks_bans.append(picks_bans)
    
    # Concatenate all DataFrames into a single DataFrame
    combined_picks_bans = pd.concat(all_picks_bans, ignore_index=True)
    
    return combined_picks_bans

In [6]:
# generate draft dataframe
picks_bans_to_read = [f'./data/{patch}/picks_bans.csv/picks_bans.csv' for patch in patch_folders]
picks_bans = read_draft(picks_bans_to_read, dota_df)
picks_bans

,hero_id,team,order,match_id
0,128.0,0.0,0.0,7276712204
1,82.0,1.0,1.0,7276712204
2,85.0,1.0,2.0,7276712204
3,61.0,0.0,3.0,7276712204
4,75.0,1.0,4.0,7276712204
...,...,...,...,...
711024,113.0,0.0,19.0,7903792349
711025,56.0,0.0,20.0,7903792349
711026,54.0,1.0,21.0,7903792349
711027,48.0,1.0,22.0,7903792349


In [7]:
picks_bans.describe()

,hero_id,team,order,match_id
count,711029.000000,711029.000000,711029.000000,7.110290e+05
mean,65.822695,0.499992,11.494173,7.591103e+09
std,37.437755,0.500000,6.920947,1.802637e+08
min,1.000000,0.000000,0.000000,7.276712e+09
25%,35.000000,0.000000,5.000000,7.439410e+09
50%,66.000000,0.000000,11.000000,7.587469e+09
75%,96.000000,1.000000,17.000000,7.746499e+09
max,138.000000,1.000000,23.000000,7.903792e+09


In [8]:
# check unique pick/ban order per match (should be all 24 orders)
picks_bans.groupby('match_id')['order'].size().describe()

count    29650.000000
mean        23.980742
std          0.427005
min         10.000000
25%         24.000000
50%         24.000000
75%         24.000000
max         24.000000
Name: order, dtype: float64

In [9]:
# drop matches that has no 24 orders
picks_bans = picks_bans.groupby('match_id').filter(
    lambda x: x['order'].nunique() == 24
    )

picks_bans.groupby('match_id')['order'].size().describe()

count    29484.0
mean        24.0
std          0.0
min         24.0
25%         24.0
50%         24.0
75%         24.0
max         24.0
Name: order, dtype: float64

In [10]:
# check unique hero id per match (should be 24 unique heroes)
picks_bans.groupby('match_id')['hero_id'].size().describe()

count    29484.0
mean        24.0
std          0.0
min         24.0
25%         24.0
50%         24.0
75%         24.0
max         24.0
Name: hero_id, dtype: float64

In [11]:
# check if there are order 0 in all matches (will be used later to determine which team is first ban)
len(picks_bans['match_id'].unique()) == len(picks_bans[picks_bans['order'] == 0])

True

### Generate sparse matrix

In [13]:
# conversion of ban/pick order to ban/pick phase (for different order on same ban phase to be interchangeable)
conversion_dict = {
    0: 1,
    1: 2,
    2: 2,
    3: 3,
    4: 4,
    5: 4,
    6: 5,
    7: 6,
    8: 7,
    9: 8,
    10: 8,
    11: 9,
    12: 10,
    13: 11,
    14: 11,
    15: 12,
    16: 12,
    17: 13,
    18: 14,
    19: 15,
    20: 15,
    21: 16,
    22: 17,
    23: 18
}

def convert_order(n):
    """Convert a number using the predefined dictionary."""
    return conversion_dict.get(n)


In [14]:
# sparse matrix
def generate_sparse_matrix(matches, picks_bans):
    num_rows = len(picks_bans['match_id'].unique())
    hero_cols = [str(int(i)) for i in picks_bans['hero_id'].unique()]
    col_names = [f'{hero_id}_{i}' for hero_id in hero_cols for i in range(1, 19)] + ['radiant_win']

    # Initialize an empty DataFrame with zeros
    sparse_draft = pd.DataFrame(
        np.zeros((num_rows, len(col_names))),
        index=picks_bans['match_id'].unique(),
        columns=col_names
    )
    
    sparse_draft['radiant_win'] = sparse_draft['radiant_win'].astype(bool)
    # sparse_draft['radiant_team_id'] = sparse_draft['radiant_team_id'].astype(int).astype(str)
    # sparse_draft['dire_team_id'] = sparse_draft['dire_team_id'].astype(int).astype(str)
    
    sparse_draft['radiant_win'] = matches['radiant_win'].reindex(sparse_draft.index, fill_value=False)
    # sparse_draft['radiant_team_id'] = matches['radiant_team_id'].reindex(sparse_draft.index, fill_value='unknown')
    # sparse_draft['dire_team_id'] = matches['dire_team_id'].reindex(sparse_draft.index, fill_value='unknown')
    
    
    # Create a list to collect the rows and columns to update
    updates = []
    
    # Iterate through each row in pick_bans and collect updates
    for _, row in picks_bans.iterrows():
        match_id = row['match_id']
        hero_order = f"{int(row['hero_id'])}_{convert_order(int(row['order']))}"
        if match_id in sparse_draft.index:
            updates.append((match_id, hero_order, 1))
    
    # Create a DataFrame from the updates and use it to update sparse_draft
    if updates:
        update_df = pd.DataFrame(updates, columns=['match_id', 'hero_order', 'value'])
        update_df.set_index(['match_id', 'hero_order'], inplace=True)
        display(update_df)
        display(update_df.unstack(level=-1)['value'].fillna(0))
        sparse_draft.update(update_df.unstack(level=-1)['value'].fillna(0))

    # add indicator to determine team with first ban
    sparse_draft = sparse_draft.merge(picks_bans.loc[picks_bans['order'] == 0, ['team', 'match_id']].set_index('match_id'), left_index=True, right_index=True, how='left')
    
    return sparse_draft

# generate sparse matrix
sparse_draft = generate_sparse_matrix(dota_df, picks_bans)
sparse_draft


value
match_id     hero_order       
7.276712e+09 128_1           1
             82_2            1
             85_2            1
             61_3            1
             75_4            1
...                        ...
7.903792e+09 113_15          1
             56_15           1
             54_16           1
             48_17           1
             6_18            1

[707616 rows x 1 columns]

hero_order,100_1,100_10,100_11,100_12,100_13,100_14,100_15,100_16,100_17,100_18,...,9_17,9_18,9_2,9_3,9_4,9_5,9_6,9_7,9_8,9_9
match_id,,,,,,,,,,,,,,,,,,,,,
7.276712e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.276715e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.276728e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.276742e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.276742e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7.903695e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7.903722e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7.903730e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,128_1,128_2,128_3,128_4,128_5,128_6,128_7,128_8,128_9,128_10,...,57_11,57_12,57_13,57_14,57_15,57_16,57_17,57_18,radiant_win,team
7276712204,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7276715296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7276728072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
7276741510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7276741939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7903694984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
7903721670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7903730404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
7903758411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0


In [15]:
# check if all team filled
sparse_draft['team'].isnull().sum()

np.int64(0)

In [16]:
# check if win indicator is filled
sparse_draft['radiant_win'].isnull().sum()

np.int64(763)

In [17]:
# drop no win indicator
sparse_draft = sparse_draft[-sparse_draft['radiant_win'].isna()]
sparse_draft

,128_1,128_2,128_3,128_4,128_5,128_6,128_7,128_8,128_9,128_10,...,57_11,57_12,57_13,57_14,57_15,57_16,57_17,57_18,radiant_win,team
7276712204,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7276715296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7276728072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
7276741510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7276741939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7903694984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
7903721670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,0.0
7903730404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0
7903758411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0


### Save outputs

In [18]:
# save utility matrix for collaborative filtering
sparse_draft.to_csv('./data/sparse_matrix.csv')

In [17]:
# save picks_bans for FIM
picks_bans.to_csv('./data/picks_bans.csv', index=False)